# LLM to Audio (testcase)

**Read the request documentation to tune your model for your application**

**Request Documentation**: https://platform.openai.com/docs/api-reference/completions/create

**OpenAI Documentation**: https://platform.openai.com/docs/quickstart?context=python

**Find your keys here**: https://platform.openai.com/api-keys

**Keep an eye on your credit usage**: https://platform.openai.com/usage

## OpenAI API Details and Hyper-Parameters

**Models**: The model you want to use (i.e. "gpt-4-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini" or "gpt-3.5-turbo")

**Message**: The message being sent (the prompt)

**Temperature**: Number between 0 and 1, can be adjusted when calling the OpenAI API to control the randomness of the output generated by the language model. It determines how "creative" or "conservative" the responses are. A temperature of 1.0 means the model generates text with standard randomness, while a lower temperature (closer to 0.0) will make it more deterministic. Suggestion: Set the temperature to a moderate value, around 0.2 to 0.5. This will give you some randomness, mimicking the variability in human input, but still maintain enough accuracy to ensure that the responses are meaningful. **In the case of our work, we held this constant at a value of 0.2 to get slight stochasticity.** 

**Frequency_penalty**: Number between -2 and 2, where higher numbers penalize new tokens based on their frequency to that point in the response. The higher the number, the lower the probability of repetition. This parameter penalizes repeated tokens in the output, making responses more diverse and creative. Suggestion: Use a low frequency penalty (e.g., 0.0 to 0.2) to reduce redundancy but still allow the model to use relevant tokens multiple times where necessary. **In the case of our work, we held this constant at a value of 0.0.**

## Set your GPT Key to your ENV

Uncomment this line to check your environment variables. That should be set up in .zshrc or .bashrc

In [128]:
# %env

In [129]:
%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_03/LLM_motion/scripts'

## Imports

In [130]:
import os
from openai import OpenAI
client = OpenAI()

import random
import time
random.seed(time.time())

from openpyxl import load_workbook

## Start Generating

### Build your prompt

In [131]:
all_states = ["stuck", "accomplished", "progressing"]

all_states_descriptions = [
    "the robot has gotten lost or is stuck behind an obstacle.",                                                 # stuck
    "the robot has successfully reached it's goal and completed its task.",                                   # accomplished
    "the robot is actively working on the task but has neither gotten stuck nor completed the task."    # progressing
]

In [132]:
gpt_assistant_prompt = "You are an expert roboticist and understand how to design communicative expressions for human-robot interaction." 

robot_morphology = "small mobile rover"

deployment_context = f"Consider a scenario where you are collaborating with a {robot_morphology} robot to navigate through a maze and find fruit."

communication_style = f"The robot uses nonverbal audio cues to communicate its task status back to the user."

parameter_01 = "Parameter 01: [Beats Per Minute (BPM), Controls the speed in which the robot plays its audio cues. If set to ‘slow’ the robot’s audio playback is set to 0.5x speed. If set to ‘medium’ the robot’s audio playback is set to 1.0x speed. If set to ‘high’ the robot’s audio playback is set to 2.0x speed, (slow, medium, fast)]"
parameter_02 = "Parameter 02: [Beats Per Loop (BPL), Controls the frequency of how many times the robot beeps per second. If set to ‘low’ the robot will beep slowly at 1 time per second. If set to ‘medium’ the robot will beep somewhat rapidly at 2 times per second. If set to ‘high’ the robot will beep rapidly 4 times per second.,(low, medium, high)]"
parameter_03 = "Parameter 03: [Pitch Bend, Controls the inflection of beeps by bending the pitch. If set to ‘downward’ the robot’s beeps will have downward inflections. If set to ‘neutral’ the robot’s beeps will remain at a neutral unchanged pitch. If set to ‘upward’ the robot’s beeps will have upward inflections.,(downward, neutral, upward)]"

# This is done to randomize the order in which the parameters are presented to GPT, to eliminate any bias for params presented first
param_list = [parameter_01, parameter_02, parameter_03]

def generate_gpt_user_prompt(robot_state_name, robot_state_description, param_list=param_list):
    
    random.shuffle(param_list) # Comment this out to remove randomness 

    return f'''
{deployment_context}

This {robot_morphology} robot is currently in state '{robot_state_name}' because {robot_state_description}

{communication_style}

Below is a list of three (3) acoustic parameters, complete with a description and a value range for each parameter. These parameters govern the characteristics of the robot's audio communication. The data in this list is in the format: [Parameter Name, Parameter Description, (Value Range)]

{param_list[0]}

{param_list[1]}

{param_list[2]}

Your Task:
From the available three (3) acoustic parameters, please select at minimum one and a maximum three of the most relevant parameters to express the robot state '{robot_state_name}'. 

Please include reasonable values within the provided value ranges for each selected motion parameter. Please only select those that you believe are most relevant. 

Please include no additional text or explanation. There should be no blank lines

Your response must keep the selected parameters in numerical order, and be in this exact format (with one line for every selected parameter):
[##, Z], (Parameter Name, Value)
[##, Z], (Parameter Name, Value) (optional)
[##, Z], (Parameter Name, Value) (optional)

## = parameter number (e.g. 02)
Z = value option, either A, B, or C.
Parameter Name = name of the selected parameter
Value = value selected for that parameter
'''

### Test Cell Below

In [133]:
current_state = all_states[0]
current_description = all_states_descriptions[0]

gpt_user_prompt = generate_gpt_user_prompt(current_state, current_description, param_list)

print(f'Your prompt for GPT is: \n\n{gpt_assistant_prompt}\n{gpt_user_prompt}')

Your prompt for GPT is: 

You are an expert roboticist and understand how to design communicative expressions for human-robot interaction.

Consider a scenario where you are collaborating with a small mobile rover robot to navigate through a maze and find fruit.

This small mobile rover robot is currently in state 'stuck' because the robot has gotten lost or is stuck behind an obstacle.

The robot uses nonverbal audio cues to communicate its task status back to the user.

Below is a list of three (3) acoustic parameters, complete with a description and a value range for each parameter. These parameters govern the characteristics of the robot's audio communication. The data in this list is in the format: [Parameter Name, Parameter Description, (Value Range)]

Parameter 01: [Beats Per Minute (BPM), Controls the speed in which the robot plays its audio cues. If set to ‘slow’ the robot’s audio playback is set to 0.5x speed. If set to ‘medium’ the robot’s audio playback is set to 1.0x speed

### Now build the OpenAI API Request

In [135]:
iteration_quantity = 80
gpt_model = "gpt-4o"                        # gpt-3.5-turbo     | Use this for dev/testing
                                            # gpt-4 / gpt-4o    | Use this when deployed, more expensive
                                            # gpt-4o-mini       | Lightweight, less expensive than gpt4o

attempt_ID = '05'

temperature_coefficient = 0.2           # Slight stochastic @ 0.2 to 0.5
frequency_penalty_coefficient = 0.0     # Lightly penalize repetition @ 0.0 to 0.2

### Multi-Iteration

In [136]:
row_counter = 1

for state_idx in range(0,len(all_states)):
    sheet_name = all_states[state_idx] + '_' + attempt_ID 
    row_counter += 1 
    selected_param_value_pairs = []

    # Load Workbook and Fill Table Entrie
    workbook_path = "./../llm_audio_testcase/llm_audio_testcase.xlsx"
    response_book = load_workbook(workbook_path)

    try: # Try to open existing sheet
        response_sheet = response_book[sheet_name]
    except KeyError:  # If ot doesn't exist. create it
        response_sheet = response_book.create_sheet(title=sheet_name)

    response_sheet["B1"] = "state"
    response_sheet["C1"] = "iteration"
    response_sheet["D1"] = "P1 BPM"
    response_sheet["E1"] = "P2 BPL"
    response_sheet["F1"] = "P3 PitchBend"

    # response_book.save(workbook_path)

    print(f'\n************************************')
    print(f'********* ROBOT STATE : {all_states[state_idx]}')
    print(f'************************************\n')


    for iteration in range(0, iteration_quantity):
        print(f'~~~~~~~~~~~~ Iteration {iteration:02d}')

        # adding to excel
        response_sheet["B"+str(iteration+2)] = all_states[state_idx]
        response_sheet["C"+str(iteration+2)] = iteration

        # generate unique prompt for each iteration
        gpt_prompt_pre_gen = generate_gpt_user_prompt(all_states[state_idx], all_states_descriptions[state_idx])
        # print(f"\n\n{gpt_prompt_pre_gen}\n\n") <- # uncomment if you want to see prompt used at each step

        # call GPT client
        completion = client.chat.completions.create(
            model=gpt_model,  # "gpt-3.5-turbo" (use this for dev/testing) or "gpt-4" (use this when deployed, more expensive)
            messages=[
                {"role": "system", "content": gpt_assistant_prompt},
                {"role": "user", "content": gpt_prompt_pre_gen}],
            temperature=temperature_coefficient,         # 0.2 or 0.5
            max_tokens=500,
            frequency_penalty= frequency_penalty_coefficient  # 0.0 or 0.2
            )
        
        # Printing result from call to GPT client
        print(completion.choices[0].message.content, '\n\n')
    
        # Now iterate and count the responses
        for line in completion.choices[0].message.content.split('\n'):
            print(f"line: {line}")

            # Check if the line is blank or too short, skip if it is
            if len(line) < 6:
                continue  # Skip to the next iteration if line is blank or too short

            try:
                collected_identifier = line[1:3] + line[5]  # Assuming the line has the required format
                print(f'Appending: {collected_identifier}')
                selected_param_value_pairs.append(collected_identifier)

                # Depending on the identifier, place the value in the right column
                if collected_identifier[0:2] == '01':
                    response_sheet["D" + str(iteration + 2)] = collected_identifier[2]

                elif collected_identifier[0:2] == '02':
                    response_sheet["E" + str(iteration + 2)] = collected_identifier[2]

                elif collected_identifier[0:2] == '03':
                    response_sheet["F" + str(iteration + 2)] = collected_identifier[2]

            except IndexError:
                print(f"Skipping line due to formatting issues: {line}")
                continue  # Skip the line if there's an IndexError (unexpected format)

        print('\n')

        print(f'selected_param_value_pairs\nLength: {len(selected_param_value_pairs)}\n{selected_param_value_pairs}')

    response_book.save(workbook_path)



************************************
********* ROBOT STATE : stuck
************************************

~~~~~~~~~~~~ Iteration 00
[01, A], (Beats Per Minute, slow)  
[02, C], (Beats Per Loop, high)  
[03, A], (Pitch Bend, downward)   


line: [01, A], (Beats Per Minute, slow)  
Appending: 01A
line: [02, C], (Beats Per Loop, high)  
Appending: 02C
line: [03, A], (Pitch Bend, downward)  
Appending: 03A


selected_param_value_pairs
Length: 3
['01A', '02C', '03A']
~~~~~~~~~~~~ Iteration 01
[01, A], (Beats Per Minute, slow)
[02, A], (Beats Per Loop, low)
[03, A], (Pitch Bend, downward) 


line: [01, A], (Beats Per Minute, slow)
Appending: 01A
line: [02, A], (Beats Per Loop, low)
Appending: 02A
line: [03, A], (Pitch Bend, downward)
Appending: 03A


selected_param_value_pairs
Length: 6
['01A', '02C', '03A', '01A', '02A', '03A']
~~~~~~~~~~~~ Iteration 02
[01, A], (Beats Per Minute, slow)
[02, A], (Beats Per Loop, low)
[03, A], (Pitch Bend, downward) 


line: [01, A], (Beats Per Minute, slow)

In [103]:
# Parse the collected data
def pair_parser(pv_pairs, printer=None):

    param01_valA = 0
    param01_valB = 0
    param01_valC = 0
    param02_valA = 0
    param02_valB = 0
    param02_valC = 0
    param03_valA = 0
    param03_valB = 0
    param03_valC = 0
    
    for pv_pair in pv_pairs:
        if pv_pair[1] == '1':
            if pv_pair[2] == 'A':
                param01_valA +=1
            if pv_pair[2] == 'B':
                param01_valB +=1
            if pv_pair[2] == 'C':
                param01_valC +=1
        if pv_pair[1] == '2':
            if pv_pair[2] == 'A':
                param02_valA +=1
            if pv_pair[2] == 'B':
                param02_valB +=1
            if pv_pair[2] == 'C':
                param02_valC +=1
        if pv_pair[1] == '3':
            if pv_pair[2] == 'A':
                param03_valA +=1
            if pv_pair[2] == 'B':
                param03_valB +=1
            if pv_pair[2] == 'C':
                param03_valC +=1

    # Sanity check
    sum = param01_valA + param02_valA + param03_valA + param01_valB + param02_valB + param03_valB + param01_valC + param02_valC + param03_valC 
    original_length = len(pv_pairs)    
    sanity_check = sum == original_length
    
    if printer:
        print(f'sanity check: {sanity_check}\n')
        print('param01_valA =', param01_valA)
        print('param01_valB =', param01_valB)
        print('param01_valC =', param01_valC)
        print('param02_valA =', param02_valA)
        print('param02_valB =', param02_valB)
        print('param02_valC =', param02_valC)
        print('param03_valA =', param03_valA)
        print('param03_valB =', param03_valB)
        print('param03_valC =', param03_valC)

In [104]:
# For the last state, lets just see if the outputs are actually giving us data in the correct format:
pair_parser(pv_pairs=selected_param_value_pairs, printer=True)

sanity check: True

param01_valA = 0
param01_valB = 40
param01_valC = 0
param02_valA = 0
param02_valB = 40
param02_valC = 0
param03_valA = 0
param03_valB = 0
param03_valC = 39


### Single Iteration

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",  # "gpt-3.5-turbo" (use this for dev/testing) or "gpt-4" (use this when deployed, more expensive)
    messages=[
        {"role": "system", "content": gpt_assistant_prompt},
        {"role": "user", "content": gpt_user_prompt}],
    temperature=0.5,
    max_tokens=500,
    frequency_penalty=0
)

print('Raw Model Output:\n\n')
print(completion.choices[0].message)

In [ ]:
print(f'Robot State: {robot_state_name}\n')

print(completion.choices[0].message.content)